# Library

In [1]:
from paddleocr import PaddleOCR, PPStructure, draw_ocr, draw_structure_result, save_structure_res
from ppstructure.recovery.recovery_to_doc import sorted_layout_boxes, convert_info_docx
from imutils.object_detection import non_max_suppression
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pdf2image import convert_from_path
from PIL import Image
import imutils
from pytesseract import Output
import pytesseract
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg
# Dòng dưới tránh bị xảy ra lỗi : Initializing libiomp5md.dll, but found libiomp5md.dll already initialized. \
# Gây ra chết kernel
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# Download and save model recognition for VietNamese

In [2]:
config = Cfg.load_config_from_name('vgg_transformer')

config['cnn']['pretrained']=False
config['device'] = 'cpu'

In [3]:
detector = Predictor(config)

Model weight C:\Users\Admin\AppData\Local\Temp\vgg_transformer.pth exsits. Ignore download!


# Read pdf and save image

In [4]:
count = 0

In [5]:
#Hàm xoay ảnh để cho text đúng
def rotate_image(path):
	image = cv2.imread(path)
	rbg = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	results = pytesseract.image_to_osd(rbg, output_type= Output.DICT)
	rotated = imutils.rotate_bound(image, results['rotate'])
	Image.fromarray(rotated).save(path)

In [6]:
#Hàm đọc file pdf và lưu lại ảnh đã xoay
def Save_Image(path):
    global count
    list_images = convert_from_path(path)
    output_dir = f'image_folder_{count}'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Lưu các hình ảnh từ list_image vào thư mục đã tạo
    for i, img in enumerate(list_images):
        image_path = os.path.join(output_dir, f'image_{i}.jpg')
        img.save(image_path)
        rotate_image(image_path)

    count+=1
    return output_dir
    print("Các hình ảnh đã được xoay và lưu vào thư mục:", output_dir)

# Structure

In [7]:
# def take_img_bb(bboxes, img, start_of_table_x, start_of_table_y):
#     box_images = []
#     for box in bboxes:
#         a, b, c, d = box
#         y_start = int(b)+start_of_table_y
#         y_end=int(d)+start_of_table_y
#         x_start=int(a)+start_of_table_x
#         x_end =int(c)+start_of_table_x
#         roi = img[y_start:y_end, x_start:x_end]
#         box_images.append(roi)
#     return box_images

In [8]:
def take_text(boxes,img):
    """Hàm lấy từng bbox trong text để tiến hành recognition lại"""
    box_images = []
    for text in boxes:
        bboxes = text['text_region']
        a,b,c,d = bboxes
        #Nguyên nhân cần phải cộng thêm 1 ít vì cách bbox có thể thiếu 1 ít,
        #để dẫn tới kết quả đúng nên cộng thêm 1 ít sẽ dẫn tới độ chính xác cao hơn
        y_start = int(a[1])-3
        y_end=int(c[1])
        x_start=int(a[0])-10
        x_end =int(c[0])+10
        roi = img[y_start:y_end, x_start:x_end]
        box_images.append(roi)
    return box_images

def sort_file_img(lis):
    """
    Hàm dùng để sắp xếp lại tên của các file ảnh
    phục vụ mục đích tìm bảng chính trong báo cáo tài chính
    """
    tmp  = []
    for i in range(len(lis)):
        tmp.append((int(lis[i].split('_')[1].split('.')[0]), lis[i]))
    tmp.sort(key = lambda x: x[0])
    sorted_lis = []
    for i in tmp:
        sorted_lis.append(i[1])
    return sorted_lis

def run_model_and_recovery(folder_path):
    config = Cfg.load_config_from_name('vgg_transformer')
    config['cnn']['pretrained']=False
    config['device'] = 'cpu'
    detector = Predictor(config)
    
    table_engine = PPStructure(show_log = False, table_model_dir ='Model\en_ppstructure_mobile_v2.0_SLANet_infer\en_ppstructure_mobile_v2.0_SLANet_infer', \
                             table_char_dict_path = r'ppocr\utils\dict\table_structure_dict.txt')
    sorted_file_name = sort_file_img(os.listdir(folder_path))
    
    for i in range(len(sorted_file_name)):
        print(f"Chạy đến ảnh {sorted_file_name[i]}")
        try:
            img_path = os.path.join(folder_path, sorted_file_name[i])
            img = cv2.imread(img_path)
            result = table_engine(img, return_ocr_result_in_table= True)
            
            check_table = False #Biến check xem có bảng tồn tại không
            for j in result:
                if j['type'] == 'table': check_table = True
            #Nếu bảng và có con dấu tồn tại thì chạy tiếp
            if check_table:
                save_folder = 'image_structure_save_1'
                for j in result:
                    text = []
                    if j['type'] == 'table':
                        """Continue vi đoạn code này đã được trực tiếp sửa trong thư viện paddleocr file predict_system.py"""
                        continue
                    else:
                        box = take_text(j['res'], cv2.imread(img_path))
                        for img in box:
                            text.append(detector.predict(Image.fromarray(img)))
                        for k in range(len(j['res'])):
                            j['res'][k]['text'] = text[k]
                save_structure_res(result, save_folder, os.path.basename(img_path).split('.')[0])

                h, w, _ = img.shape
                res = sorted_layout_boxes(result, w)
                convert_info_docx(img, res, save_folder, os.path.basename(img_path).split('.')[0])
                
            print(f"Ảnh {img_path} ok")
        except:
            print(f"Ảnh {img_path} có lỗi không lưu được table")
    print("Đã OCR và xuất ra file excel, docx thành công")

In [9]:
def OCR(pdf_path):
    folder_path = 'image_folder_0' #Save_Image(pdf_path)
    run_model_and_recovery(folder_path)

In [10]:
for file in os.listdir('PDF'):
    OCR(os.path.join('PDF', file))

Model weight C:\Users\Admin\AppData\Local\Temp\vgg_transformer.pth exsits. Ignore download!
Chạy đến ảnh image_0.jpg
Ảnh image_folder_0\image_0.jpg ok
Chạy đến ảnh image_1.jpg
Ảnh image_folder_0\image_1.jpg ok
Chạy đến ảnh image_2.jpg
Ảnh image_folder_0\image_2.jpg ok
Chạy đến ảnh image_3.jpg
Ảnh image_folder_0\image_3.jpg ok
Chạy đến ảnh image_4.jpg
Ảnh image_folder_0\image_4.jpg ok
Chạy đến ảnh image_5.jpg
Model weight C:\Users\Admin\AppData\Local\Temp\vgg_transformer.pth exsits. Ignore download!
[2024/06/06 13:23:04] ppocr INFO: docx save to image_structure_save_1\image_5_ocr.docx
Ảnh image_folder_0\image_5.jpg ok
Chạy đến ảnh image_6.jpg
Model weight C:\Users\Admin\AppData\Local\Temp\vgg_transformer.pth exsits. Ignore download!
[2024/06/06 13:23:46] ppocr INFO: docx save to image_structure_save_1\image_6_ocr.docx
Ảnh image_folder_0\image_6.jpg ok
Chạy đến ảnh image_7.jpg
Model weight C:\Users\Admin\AppData\Local\Temp\vgg_transformer.pth exsits. Ignore download!
[2024/06/06 13:24:35